### Домашнее задание к лекции «Основы парсинга и работы с API»


### Оглавление
  
1. [Задание 1](#task1)

2. [Задание 2](#task2)  

###### 1. Задание 1 <a id='task1'></a>

### Обязательная часть


Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [23]:
# импортируем все необходимые библиотеки
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [24]:
# формируем необходимые стартовые переменные
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг', 'it']
habr_news = pd.DataFrame()
# Делаем запрос
req = requests.get(URL)
# Преобразуем с помощью BeautifulSoup
soup = BeautifulSoup(req.text, "html.parser")
# делаем выборку по требуемому классу
posts_all = soup.find_all('div', class_='tm-article-snippet')
# Запускаем цикл перебора постов
for post in posts_all:
    post_text = post.find('div', class_='article-formatted-body').text
    # переводим текст поста в нижний регистр
    post_lower = post_text.lower()
    # запускаем цикл перебора поисковых слов в тексте( в нижнем регистре)
    for w in KEYWORDS:
        if w.lower() in post_lower:
            # вычленяем дату
            date = pd.to_datetime(post.find(
                'span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True)
            # вычленяем заголовок
            title = post.find(
                'h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
            # вычленяем адрес ссылки
            url_ = post.find('a').get('href')
            # формируем полноценные ссылки
            all_url = 'https://habr.com/' + url_
            # готовим строку для датафрейма
            row = {'date': date, 'title': title, 'url': all_url}
            # добавляем строку к датафрейму (делаем сразу адекватную индексацию)
            habr_news = pd.concat([habr_news, pd.DataFrame([row])], ignore_index=True)
# меняем заголовки столбцов
habr_news.columns = ['дата', 'заголовок', 'ссылка']
habr_news

,дата,заголовок,ссылка
0,2021-08-01 08:39:52+00:00,Миграция платежной платформы в облако: Зачем и...,https://habr.com//ru/users/Vladimir_Kuiantsev/
1,2021-07-31 19:09:22+00:00,iQunix A80 Explorer: беспроводная клавиатура в...,https://habr.com//ru/users/Rider_GT/
2,2021-07-31 15:21:32+00:00,Как spring-kafka обрабатывает сообщения и не м...,https://habr.com//ru/users/dm_aq/
3,2021-07-31 15:18:51+00:00,"Tweet-a-pot, или Как я через Twitter кофе варил",https://habr.com//ru/users/KD637/
4,2021-07-31 13:09:41+00:00,Попытка создать java Framework для телеграм ботов,https://habr.com//ru/users/slyrack0/


###### 1. Задание 1 

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [25]:
# формируем необходимые стартовые переменные
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг', 'it']
habr_news_ = pd.DataFrame()
# Делаем запрос
req = requests.get(URL)
# Преобразуем с помощью BeautifulSoup
soup = BeautifulSoup(req.text, "html.parser")
# делаем выборку всех постов
posts_all = soup.find_all('div', class_='tm-article-snippet')
# делаем выборку по требуемому классу
posts_intro = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'), posts_all))
# извлекаем ссылку каждого поста
url_list = list(map(lambda x: x.find('a').get('href'), posts_intro))
# формируем полноценные ссылки каждого поста
all_url_ = list(map(lambda x: 'https://habr.com' + x, url_list))
all_url_

['https://habr.com/ru/company/ruvds/blog/566522/',
 'https://habr.com/ru/post/570716/',
 'https://habr.com/ru/post/570696/',
 'https://habr.com/ru/post/570710/',
 'https://habr.com/ru/post/570642/',
 'https://habr.com/ru/post/570690/',
 'https://habr.com/ru/company/selectel/blog/570020/',
 'https://habr.com/ru/company/simbirsoft/blog/570514/',
 'https://habr.com/ru/post/570612/',
 'https://habr.com/ru/company/audiomania/blog/570550/',
 'https://habr.com/ru/post/570676/',
 'https://habr.com/ru/post/570674/',
 'https://habr.com/ru/company/skillfactory/blog/570596/',
 'https://habr.com/ru/post/570666/',
 'https://habr.com/ru/post/569996/',
 'https://habr.com/ru/company/ruvds/blog/569358/',
 'https://habr.com/ru/post/570664/',
 'https://habr.com/ru/post/569530/',
 'https://habr.com/ru/post/570660/',
 'https://habr.com/ru/company/audiomania/blog/570630/']

In [28]:
# запускаем цикл перебора ссылок каждого поста
for link in all_url_:
    # Делаем запрос и сразу Преобразуем с помощью BeautifulSoup
    soup_ = BeautifulSoup(requests.get(link).text, 'html.parser')
    # делаем задержку по времени
    time.sleep(0.3)
    # извлекаем текст
    p_text = soup_.find('div', class_='tm-article-body').text
    #  переводим в нижний регистр текст
    p_lower = p_text.lower()
    # запускаем цикл перебора поисковых слов в тексте( в нижнем регистре)
    for w in KEYWORDS:
        if w.lower() in p_lower:
            # извлекаем дату
            date = pd.to_datetime(soup_.find(
                'span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True)
            # извлекаем заголовок
            title = soup_.find(
                'h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text
            # готовим строку для датафрейма
            row = {'date': date, 'title': title, 'url': link, 'text': p_text}
            # добавляем строку к датафрейму (делаем сразу адекватную индексацию)
            habr_news_ = pd.concat(
                [habr_news_, pd.DataFrame([row])], ignore_index=True)
            # чтобы исключить повторное появление в таблице этого поста, в случае нахождения в нем нескольких ключевых слов, прерываем этот цикл
            break
# меняем заголовки столбцов
habr_news_.columns = ['дата', 'заголовок', 'ссылка', 'текст статьи']
habr_news_

,дата,заголовок,ссылка,текст статьи
0,2021-08-01 10:01:01+00:00,MMT для UBI: как современная теория денег объя...,https://habr.com/ru/company/ruvds/blog/566522/,MMT или современная теория денег (Modern Monet...
1,2021-08-01 09:31:25+00:00,Процессор Эльбрус — почему это тупик для разви...,https://habr.com/ru/post/570716/,Последние месяцы ознаменовались бурными батали...
2,2021-08-01 09:05:22+00:00,"Горячий, умный, рогатый: разработка анемометра...",https://habr.com/ru/post/570696/,Приключения железного стартапа в РоссииЗдравст...
3,2021-08-01 08:39:52+00:00,Миграция платежной платформы в облако: Зачем и...,https://habr.com/ru/post/570710/,Источник: Telegram: @tproger_officialПривет! Н...
4,2021-08-01 06:43:36+00:00,Что такое функциональное программирование?,https://habr.com/ru/post/570642/,Эта статья является переводом материала «What ...
5,2021-07-31 20:51:52+00:00,"Володя, где мой компьютер? Проблемы школьной и...",https://habr.com/ru/post/570690/,"Привет, дорогой читатель. Да, сейчас будет ста..."
6,2021-07-31 19:09:22+00:00,iQunix A80 Explorer: беспроводная клавиатура в...,https://habr.com/ru/company/selectel/blog/570020/,\r\nМы довольно часто публикуем обзоры разных ...
7,2021-07-31 18:13:31+00:00,Flutter: открытие виджетов ключами,https://habr.com/ru/company/simbirsoft/blog/57...,Всем привет! Мы продолжаем делиться полезными ...
8,2021-07-31 18:12:59+00:00,Теория графов. Часть третья (Представление гра...,https://habr.com/ru/post/570612/,"Все, что познается, имеет число, ибо невозможн..."
9,2021-07-31 18:06:29+00:00,Три предметно-ориентированных языка программир...,https://habr.com/ru/company/audiomania/blog/57...,"В прошлый раз мы рассказывали о книгах, которы..."


###### 2. Задание 2 <a id='task2'></a>

### Обязательная часть



Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [30]:
# формируем необходимые стартовые переменные

# в случае нахождения в списке нескольких адресов одновременно, сервис выдает информацию по ним в перемешанном виде, не позволяющем отсортировать информацию по отдельным адресам
EMAIL = ['yyy@y.com']
#EMAIL = ['xxx@x.ru']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/'
}
EMAIL_S={}
EMAIL_S['emailAddresses'] = EMAIL
# делаем post запрос
res = requests.post(URL, headers=HEADERS, json=EMAIL_S)
# преобразуем полученную информацию (Конвертируем из json в словарь, разворачиваем столбцы) и формируем датафрейм
df = pd.DataFrame.from_dict(res.json()['breaches'], orient='index')
# делаем адекватную индексацию
df = pd.concat([df], ignore_index=True)
# отбираем нужные столбцы
df1 = df[['breachId', 'publishDate', 'site', 'description']]
# переименовываем столбцы
df1.columns = ['идентификатор', 'дата утечки',
               'источник утечки', 'описание утечки']
df1

,идентификатор,дата утечки,источник утечки,описание утечки
0,2,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
1,3,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
2,15,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
3,41,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
4,3520,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
5,3587,2017-03-01T00:00:00Z,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."
6,3669,2017-03-24T00:00:00Z,youku.com,Youku is a large Chinese video content company...
7,13094,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
8,13662,2018-02-18T00:00:00Z,netlog.com,Netlog (formerly known as Facebox and Bingbox)...
9,16768,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."


###### 2. Задание 2

### Дополнительная часть (необязательная)



Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [31]:
#Сервисный ключ доступа


token = ''

In [32]:
# формируем необходимые стартовые переменные
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get'
VERSION = '5.58'
DOMAIN = 'netology'
TOKEN = token
COUNT = 50
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': DOMAIN,
    'count': COUNT
}

In [33]:
# делаем запрос
res = requests.get(NEWSFEED_REQUEST, params)
# преобразуем полученную информацию (Конвертируем из json в словарь и формируем датафрейм)
df = pd.DataFrame.from_dict(res.json()['response']['items'])
# отбираем нужные столбцы
df_res = df[['date', 'text']]
# переименовываем столбцы
df_res.columns = ['дата поста', 'текст поста']
df_res

,дата поста,текст поста
0,1627653606,"Компьютеры появились ещё в прошлом столетии, н..."
1,1627803420,"Учиться тоже нужно уметь 🤔 \n \nСобрали книги,..."
2,1627715880,"У диджитал-профессий есть свои особенности, о ..."
3,1627635770,В конце каждого месяца заботливо собираем подб...
4,1627573380,Работа в диджитал ≠ лежать под пальмой где-ниб...
5,1627546980,"Продолжаем снимать новый формат видео, в котор..."
6,1627480805,Ещё несколько дней и начнётся последний месяц ...
7,1627458960,👩‍🎨 Запустили бесплатный курс «Как стать дизай...
8,1627400400,"Когда действительно любишь работу, часто бывае..."
9,1627373520,Куда развивать стартап — отвечает предпринимат...
